<a href="https://colab.research.google.com/github/Sumitkevlani/LLM/blob/main/Langchain_codebasics_cuisine_name_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import userdata
openai_key = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install langchain openai langchain_openai langchain_core langchain-huggingface

In [79]:
from huggingface_hub import login
login()

In [19]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [42]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=64,
    do_sample=False,
    repetition_penalty=1.03,
)

In [43]:
chat = ChatHuggingFace(llm=llm, verbose=True)

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [93]:
region = "Mexican"
messages = [
    ("system", "You are an expert in generating names for the restaurants according to the region."),
    ("human", f"Give me names of some {region} restaurants."),
]

In [57]:
chat.invoke(messages,{"region":"Indian"})

AIMessage(content="Certainly! Here's a list of some well-regarded Indian restaurants across the United States. Please note that menus and service might vary, so it's always a good idea to check the latest reviews or offerings:\n\n1. **Tabla** - Chicago, IL\n\n2. **Amigos Indian Bistro** - Dallas, TX\n\n3. **India Pale Ale & Shamata's** - Atlanta, GA\n", response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=13, total_tokens=113), 'model': '', 'finish_reason': 'length'}, id='run-23ea1856-ce1a-44dd-8a9c-cff981327cd3-0')

In [86]:
mistral_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    max_new_tokens=64,
    do_sample=False,
    repetition_penalty=1.03,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [87]:
mistral_chat = ChatHuggingFace(llm=llm, verbose=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [88]:
mistral_chat.invoke(messages,{"region":"Indian"})

AIMessage(content="Certainly! Here's a list of some well-regarded Indian restaurants across the United States. Please note that menus and service might vary, so it's always a good idea to check the latest reviews or offerings:\n\n1. **Tabla** - Chicago, IL\n\n2. **Amigos Indian Bistro** - Dallas, TX\n\n3. **India Pale Ale & Shamata's** - Atlanta, GA\n", response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=13, total_tokens=113), 'model': '', 'finish_reason': 'length'}, id='run-09a810a5-da23-42a1-ab71-66cda5aece51-0')

In [89]:
llama_llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-2-7b",
    task="text-generation",
    max_new_tokens=64,
    do_sample=False,
    repetition_penalty=1.03,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [90]:
llama_chat = ChatHuggingFace(llm=llm, verbose=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [94]:
llama_chat.invoke(messages)

AIMessage(content='Here are some well-known Mexican restaurants from various cities around the world:\n\n\n**United States:**\n\n1. El Jardin Cocina Mexicana - Austin, Texas\n\n2. Casa Bonita - Denver, Colorado\n\n3. Casa de Luz - New York City, NY\n\n4. Los Cabos - Miami, Florida\n\n5. Molé - Los Angeles, California\n\n\n**Mexico:**\n\n', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=13, total_tokens=113), 'model': '', 'finish_reason': 'length'}, id='run-dab0ca1b-673c-47c5-803b-5a90fcd7c18c-0')

In [95]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

In [142]:
template = """There is a restaurant in {region}. Its name is """
prompt_template = PromptTemplate(input_variables=["region"], template=template)
name_chain = LLMChain(prompt=prompt_template, llm=mistral_llm,output_key="restaurant_name")


In [144]:
name_chain.run("India")

'100% Pure Vegetarian Restaurant. The restaurant owner is Mr. Rajnikanth. The restaurant is situated in Mumbai, Maharashtra. Mr. Rajnikanth is also the owner of the restaurant. It is very famous because it has the best food in the world.'

In [148]:
items_name_template = """The food items at {restaurant_name} restaurant are very tasty. Some of them are """
items_name_prompt_template = PromptTemplate(input_variables=["restaurant_name"], template=items_name_template)
items_name_chain = LLMChain(prompt=items_name_prompt_template, llm=mistral_llm,output_key="menu_items")

In [149]:
items_name_chain.run("Great Indian Restaurant")

'1. Dal makhani 2. Chicken tikka 3. Chicken biriyani 4. Chicken korma 5. Aloo paratha.\n\nThe first thing to do is to decide what type of food you want to eat. There are a lot'

In [138]:
from langchain.chains import SimpleSequentialChain

overall_chain = name_chain | items_name_chain

In [150]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[name_chain,items_name_chain])

In [151]:
overall_chain.run("Mexico")

"3m chicken, spicy chicken, fish tacos, chicken fajitas, etc. You can also order a combination of these items. And you can choose the sauce as well.\n\nOne of the best restaurants is 3M. It's located at Tepoztlan, a small"

In [152]:
overall_chain.run("India")

'100% Pure Vegetarian Restaurant. The owner of the restaurant is Mr. Rajnikanth. He is also the owner of the restaurant.\n\n## What is 100% Pure Vegetarian Restaurant?\n\nThe 100% Pure Vegetarian'

In [153]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[name_chain,items_name_chain],
    input_variables=["region"],
    output_variables=["restaurant_name","menu_items"],
    verbose=True,
)

In [154]:
chain({"region":"Arabic"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'region': 'Arabic',
 'restaurant_name': 'حاوية الجبان, which literally means “the boat of the Camel”.\n\nI don’t know how it got its name. I don’t know anything about it except that it is there.\n\nThere are so many restaurants in Dubai and around the world,',
 'menu_items': '100% Halal and some of them are not.\n\nI’m not a regular diner in this restaurant, but I have been to this place with my friends and family. This restaurant has a very nice ambiance. The restaurant serves very tasty and delicious food.\n\nThe food is'}

In [155]:
chain({"region":"American"})



> Entering new SequentialChain chain...

> Finished chain.


{'region': 'American',
 'restaurant_name': '333 Pacific. It is a small and famous restaurant. You can get the Chinese food, the Indian food, the Thai food, the Italian food, the Mexican food, the American food. The French food. The Japanese food. The Korean food. The Greek food. The food from Africa. The food',
 'menu_items': '333 Pacific, 333 Pacific-USA, 333 Pacific-China, 333 Pacific-Italian.\n\nThe food restaurant is one of the most important things in my life. I have been working in this restaurant for almost a year. This restaurant has a'}